In [1]:
from MixedAutoencoder import Mixer, MixedAutoencoder
import MixedAutoencoder
from DataCleaning import *
import DataCleaning
import pandas as pd
import random
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pywebio.output import *
from pywebio.input import *
from pywebio import start_server
import io
matplotlib.use('agg')
seed = 42
MixedAutoencoder.setRandom(seed)
DataCleaning.setRandom(seed)
base_path = "."
mixer = Mixer(base_path)

num_sets = 4
latent_dim = 8
model_shape = []
encoder = "demo"
label = f'demo_{num_sets}_{latent_dim}_[{"_".join([str(s) for s in model_shape])}]'

model, extra = mixer.load_from_label(label)
model_3d, _extra = mixer.load_from_label(label+"_3d")
#print(len(extra["columns"][encoder]))
key_list = extra["key_list"]
columns = extra["columns"]

sets = [k for k in key_list if k != encoder]

In [2]:

data = pd.read_csv(f'{base_path}/data/16PF/data.csv', sep="\t")
data = clean_data(data)
data = data.sample(frac=1)
split_data = split(data, columns)
train, test = make_train_test(split_data, 0.8)
full_columns = {"main": list(data), "main2": list(data)}
full_data = split(data, full_columns)
full_train, full_test = make_train_test(full_data, 0.8)

In [3]:
model.show_subtotal_accuracy(["set1", "set2", "set3", "set4"], test)

Binary accuracy: 0.40778210759162903


In [4]:
model.show_subtotal_binary_accuracy(["set1", "set2", "set3", "set4"], test)

Binary accuracy: 0.7905327476318705


In [5]:
model.eval_set(test)

('set1', 'set1')
221/221 [==============================] - 0s 530us/step - loss: 0.6940
compared to average: 1.5601943501759679
('set1', 'set2')
221/221 [==============================] - 0s 517us/step - loss: 0.9193
compared to average: 1.5142659627953745
('set1', 'set3')
221/221 [==============================] - 0s 530us/step - loss: 0.9656
compared to average: 1.4522765837104072
('set1', 'set4')
221/221 [==============================] - 0s 557us/step - loss: 0.9663
compared to average: 1.4666161044837516
('set1', 'demo')
221/221 [==============================] - 0s 521us/step - loss: 0.8550
compared to average: 1.5450980392156863
('set2', 'set1')
221/221 [==============================] - 0s 552us/step - loss: 0.9266
compared to average: 1.5601943501759679
('set2', 'set2')
221/221 [==============================] - 0s 537us/step - loss: 0.6467
compared to average: 1.5142659627953745
('set2', 'set3')
221/221 [==============================] - 0s 521us/step - loss: 0.9088
compared

In [6]:
autoencoder = mixer.make_new({"main": [], "main2": []}, latent_dim, {"main": len(full_columns["main"]), "main2": len(full_columns["main2"])})
settings = {
    "training": [["$all", "$all", True, True]]#[[[k1], [k2 for k2 in keys if k1 != k2], True, True] for k1 in keys],
    #"encoder_proximity_training": [["$all", True]],
    #"plot": [True, 3, [0, 1, 2]]
}
autoencoder.train_set(full_train, 25, autoencoder.make_train_config(settings = settings), batch_size = 64, verbose=True)

epoch 0
442/442 [==============================] - 2s 4ms/step - loss: 1.1674 - val_loss: 1.3654 - accuracy: 0.2778
epoch 1
442/442 [==============================] - 1s 2ms/step - loss: 1.0664 - val_loss: 1.2752 - accuracy: 0.3153
epoch 2
442/442 [==============================] - 1s 2ms/step - loss: 1.0072 - val_loss: 1.2163 - accuracy: 0.3215
epoch 3
442/442 [==============================] - 1s 2ms/step - loss: 0.9562 - val_loss: 1.1674 - accuracy: 0.3314
epoch 4
442/442 [==============================] - 1s 2ms/step - loss: 0.9107 - val_loss: 1.1278 - accuracy: 0.3444
epoch 5
442/442 [==============================] - 1s 2ms/step - loss: 0.8737 - val_loss: 1.0961 - accuracy: 0.3552
epoch 6
442/442 [==============================] - 1s 2ms/step - loss: 0.8457 - val_loss: 1.0722 - accuracy: 0.3678
epoch 7
442/442 [==============================] - 1s 2ms/step - loss: 0.8243 - val_loss: 1.0531 - accuracy: 0.3742
epoch 8
442/442 [==============================] - 1s 2ms/step - loss: 0

In [8]:
autoencoder.show_total_accuracy(full_test)
autoencoder.show_total_binary_accuracy(full_test)

total accuracy: 0.44711536169052124
Binary accuracy: 0.8344270757328582
